In [ ]:
USE_SERVER = True
if USE_SERVER:
    !pip install elasticsearch
    !pip install elasticsearch_dsl
    !pip install pymed
    !pip install gensim
    !pip install torchsummary

In [ ]:
from __future__ import print_function
import argparse
import numpy as np
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import utils
import os
from utils import PROJECT_ROOT, DATA_PATH
import yuval_module.paper_source as PaperSource

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

import py_4.get_mesh_vec as get_mesh_vec
import py_3.sim_matrix_3 as sim_matrix_3
import py_4.get_all_features as get_all_features 

from torch.utils.data import Dataset, DataLoader

from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import time

from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

In [ ]:
mesh_embed=get_mesh_vec.MeshEmbeddings(PROJECT_ROOT + "data/mesh_data/MeSHFeatureGeneratedByDeepWalk.csv")

In [ ]:
FILE = "enriched_labeled_dataset_large" 
if os.path.exists(PROJECT_ROOT + DATA_PATH + FILE):
    print("READING FROM LOCAL")
    if FILE.split(".")[1] == "json":
        df = pd.read_json(PROJECT_ROOT + DATA_PATH + FILE)
    else:
        df = pd.read_csv(PROJECT_ROOT + DATA_PATH + FILE)
    #ps = PaperSource()
else:
    print("PULLING FROM S3")
    ps = sim_matrix_3.load_dataset(FILE)
    df = ps.get_dataset()

df.drop(columns=["last_author_country"],inplace=True)
df.rename(columns={'ORG_STATE':'last_author_country'},inplace=True)

print("FILE PULLED")

In [ ]:
cd ..

### HYPERPARAMETER OPTIMIZATION ###

In [ ]:
#settings

BATCH_SIZE= 32
EPOCHS = 30
cuda = torch.cuda.is_available()
seed = 42
log_interval = 10
num_workers = 2

#check for cuda
device = torch.device('cuda' if cuda else 'cpu')

torch.manual_seed(seed)
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

device

In [ ]:
MODEL_PATH = pass

In [ ]:
### TODO: GET THE VAL DATA ###

# Get the VAE MODEL

model = torch.load(MODEL_PATH)
model.eval()

#Get the VAE Feat
vae_feat = None

In [ ]:
db__eps = np.linspace(.40,.60,40)

best_eps = None
best_f1 = 0.0

for db_eps in db__eps:
    df_all_cases = []

    for auth in dict_auth:
        df_auth = df[df['last_author_name'] == auth] 
        data = ToyDS(df, selection_test, vae_features = vae_feat.__getvae__())
        data = torch.from_numpy(data).to(device,dtype=torch.float32)
        _, mu, __ = model(data)
        latent_feat = mu.detach().cpu().numpy()
        df_all_cases.append([df_auth,latent_feat])
        
    y_hat_comb = []

        
    for case in df_all_cases:
        df_clus, latent_feat = case
        y_hat = DBS(eps=db_eps,min_samples=1,metric="euclidean").fit(latent_feat)
        df_clus = df_clus[["pmid","PI_IDS"]]
        df_clus['cluster_pred'] = y_hat.labels_
        y_hat_comb.append(df_clus)

        
    f1, precision, recall, df_eval = metric_eval_2.get_metrics_many(y_hat_comb)

    if f1 > best_F1:
        best_f1 = f1
        best_eps = db_eps

In [ ]:
print(best_eps)
print(best_f1)

### GET SCORES FROM USE CASES

In [ ]:
usecase = list(pd.read_csv("../data/test_set_author_names.csv").drop(columns=["Unnamed: 0"])["0"])
df_usecase = df[df['last_author_name'].isin(usecase)]

In [ ]:
total_metric = []
use_cases = ['2_da_same','2_da_dif','3_da', '1_da','mix_bag']

for case in use_cases:
    y_hat_comb, num_cases, num_papers = db_scan_3.db_multiple(ps,df,scaler = scaler, authors=usecase,use_case=case,weights=weights,bias=bias,epsilon=epsilon)
    f1, precision, recall, df_eval = metric_eval_2.get_metrics_many(y_hat_comb)
    total_metric.append([case,f1,precision,recall,df_eval, num_cases, num_papers])